# Perturbations

Generating the anticommonsensical dataset

#### 1. Data Preparation


In [228]:
import pandas as pd
import json

dataset_path = "../data/cladder/cladder-v1-q-commonsense.json"
with open(dataset_path, "r") as f:
    data = json.load(f)
    
df = pd.DataFrame(data)
df.rename(columns={'given_info': 'info'}, inplace=True)

df = df[df['meta'].apply(lambda x: x.get('query_type') != 'backadj')].reset_index(drop=True)


In [229]:
df.iloc[0]['question']


'Will alarm set by husband increase the chance of ringing alarm?'

In [230]:
df.head(2)

,question_id,desc_id,info,question,answer,meta,reasoning
0,0,alarm-mediation-ate-model0-spec0-q0,"For husbands that don't set the alarm, the pro...",Will alarm set by husband increase the chance ...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
1,1,alarm-mediation-ate-model0-spec0-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...


In [231]:
df.iloc[100]['meta']

{'story_id': 'alarm',
 'graph_id': 'mediation',
 'mediators': ['V2'],
 'polarity': False,
 'groundtruth': 0.35250941023943905,
 'query_type': 'nde',
 'rung': 3,
 'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]',
 'given_info': {'p(Y | X, V2)': [[0.07539329207269328, 0.5003865412696569],
   [0.3886361970461835, 0.8599843795718491]],
  'p(V2 | X)': [0.8470836315848196, 0.289175632189055]},
 'estimand': '\\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]',
 'treatment': 'X',
 'outcome': 'Y',
 'model_id': 22}

In [232]:
index = 100

print('Info: ', df.iloc[index]['info'])
print('Question: ', df.iloc[index]['question'])
print('Answer: ', df.iloc[index]['answer'])
print('Graph ID: ', df.iloc[index]['meta']['graph_id'])
print('Query type: ', df.iloc[index]['meta']['query_type'])
print('Rung: ', df.iloc[index]['meta']['rung'])
print('Formal form: ', df.iloc[index]['meta']['formal_form'])
print('Reasoning: ' , df.iloc[index]['reasoning'])

Info:  For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 50%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 39%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 85%. For husbands that set the alarm, the probability of alarm set by wife is 29%.
Question:  If we disregard the mediation effect through wife, would husband negatively affect alarm clock?
Answer:  no
Graph ID:  mediation
Query type:  nde
Rung:  3
Formal form:  E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]
Reasoning:  {'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'step3': '\\sum_{V2=v} P(V2=v|X=0)*[P(Y

In [233]:
df['meta'][3]

{'story_id': 'alarm',
 'graph_id': 'mediation',
 'mediators': ['V2'],
 'polarity': False,
 'groundtruth': -0.2305349321780112,
 'query_type': 'nie',
 'rung': 3,
 'formal_form': 'E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]',
 'given_info': {'p(Y | X, V2)': [[0.08430222457648505, 0.5394610521458689],
   [0.4061509701126924, 0.8620283206949241]],
  'p(V2 | X)': [0.7416866188819116, 0.23519324071521291]},
 'estimand': '\\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X = 1) − P(V2 = v | X = 0)]',
 'treatment': 'X',
 'outcome': 'Y',
 'model_id': 0}

In [234]:
df_new = df.copy()
meta_df = df_new['meta'].apply(pd.Series)
meta_df
df_new = pd.concat([df_new, meta_df], axis = 1)
df_new = df_new.drop('meta', axis = 1)
df_new.rename(columns={'given_info': 'given_info_meta', 'given_info': 'given_info'}, inplace=True)

In [235]:
df_new['query_type'].unique()

array(['ate', 'ett', 'nie', 'nde', 'marginal', 'correlation', 'exp_away',
       'collider_bias', 'det-counterfactual'], dtype=object)

In [236]:
df_new.columns

Index(['question_id', 'desc_id', 'info', 'question', 'answer', 'reasoning',
       'story_id', 'graph_id', 'treated', 'result', 'polarity', 'groundtruth',
       'query_type', 'rung', 'formal_form', 'given_info', 'estimand',
       'treatment', 'outcome', 'model_id', 'mediators', 'baseline', 'collider',
       'action'],
      dtype='object')

In [237]:
df_sampled = df_new.sample(n = 1000, random_state=25)
print(len(df_sampled))

1000


In [238]:
print(df_sampled.head(10))

      question_id                                            desc_id  \
297           915  firing_employee-diamondcut-marginal-model79-sp...   
4914        15784       vaccine_kills-diamond-nie-model4119-spec5-q0   
7222        23367  smoke_birthWeight-arrowhead-ett-model4780-spec...   
3318        10711  smoke_birthWeight-arrowhead-nde-model936-spec6-q1   
7091        22933  simpson_kidneystone-confounding-ett-model4739-...   
3787        12133   smoking_tar_cancer-chain-ate-model1046-spec26-q1   
3466        11200  smoking_frontdoor-frontdoor-nie-model973-spec1...   
2392         7802  nature_vs_nurture-arrowhead-nie-model677-spec1...   
2716         8852  orange_scurvy-chain-correlation-model752-spec2-q0   
8306        25842  orange_scurvy-chain-det-counterfactual-model33...   

                                                   info  \
297   The overall probability of manager signing the...   
4914  For unvaccinated individuals, the probability ...   
7222  For infants with nonsmok

In [239]:
column_names = ['answer', 'query_type', 'answer', 'graph_id', 'rung', 'query_type', 'story_id', 'polarity']

for column_name in column_names:
    print(df_sampled[column_name].value_counts())
    print(df_new[column_name]. value_counts())
    print('----------------------------------')

answer
no     504
yes    496
Name: count, dtype: int64
answer
yes    4345
no     4345
Name: count, dtype: int64
----------------------------------
query_type
marginal              209
ate                   174
correlation           174
ett                   138
det-counterfactual     95
nie                    92
nde                    73
collider_bias          23
exp_away               22
Name: count, dtype: int64
query_type
marginal              1702
ate                   1518
correlation           1518
ett                   1288
nie                    874
det-counterfactual     870
nde                    552
exp_away               184
collider_bias          184
Name: count, dtype: int64
----------------------------------
answer
no     504
yes    496
Name: count, dtype: int64
answer
yes    4345
no     4345
Name: count, dtype: int64
----------------------------------
graph_id
mediation      197
arrowhead      188
confounding    106
diamond        105
IV             102
chain           

In [240]:
index = 4

print('Info: ', df_sampled.iloc[index]['info'])
print('Question: ', df_sampled.iloc[index]['question'])
print('Answer: ', df_sampled.iloc[index]['answer'])  p
print('Graph ID: ', df_sampled.iloc[index]['graph_id'])
print('Query type: ', df_sampled.iloc[index]['query_type'])
print('Rung: ', df_sampled.iloc[index]['rung'])
print('Formal form: ', df_sampled.iloc[index]['formal_form'])
print('Reasoning: ' , df_sampled.iloc[index]['reasoning'])

Info:  For patients who have small kidney stones and not receiving treatment, the probability of recovery is 88%. For patients who have small kidney stones and receiving treatment, the probability of recovery is 72%. For patients who have large kidney stones and not receiving treatment, the probability of recovery is 16%. For patients who have large kidney stones and receiving treatment, the probability of recovery is 4%. The overall probability of large kidney stone is 41%.
Question:  For patients receiving treatment, would it be less likely to see recovery if the patient had received no treatment?
Answer:  no
Graph ID:  confounding
Query type:  ett
Rung:  3
Formal form:  E[Y_{X = 1} - Y_{X = 0} | X = 1]
Reasoning:  {'step0': 'Let V1 = kidney stone size; X = treatment; Y = recovery.', 'step1': 'V1->X,V1->Y,X->Y', 'step2': 'E[Y_{X = 1} - Y_{X = 0} | X = 1]', 'step3': '\\sum_{V1=v} P(V1=v|X=1)*[P(Y=1|V1=v,X=1) - P(Y=1|V1=v, X=0)]', 'step4': 'P(Y=1 | V1=0, X=0) = 0.88\nP(Y=1 | V1=0, X=1)

#### 2. Apply perturbations with a LLM

In [243]:
df_cladder = df_sampled.copy()

def add_column(df, model_name):   
    if model_name not in df.columns: 
        df[model_name] = None
    else:
        print(model_name + " already exists in Dataframe!")


column_names = ['anticommonsensical_info', 'anticommonsensical_question', 'nonsensical_info', 'nonsensical_question']


for c in column_names:
    add_column(df_cladder, c)    

df_cladder = df_cladder.reset_index(drop=True)

In [265]:
index = 950

variables = df_cladder.iloc[index]['reasoning']['step0']
info = df_cladder.iloc[index]['info']
question = df_cladder.iloc[index]['question']

print(variables, info, question)


Let V1 = age; X = hospital costs; Y = recovery. The overall probability of high hospital bill is 44%. For patients who pay a low hospital bill, the probability of recovery is 28%. For people who pay a high hospital bill, the probability of recovery is 64%. Is recovery less likely than non-recovery overall?


In [266]:
'Variables: ' + variables + '\n' + 'Info: ' + info + '\n' + 'Question: ' + question + '\n'

'Variables: Let V1 = age; X = hospital costs; Y = recovery.\nInfo: The overall probability of high hospital bill is 44%. For patients who pay a low hospital bill, the probability of recovery is 28%. For people who pay a high hospital bill, the probability of recovery is 64%.\nQuestion: Is recovery less likely than non-recovery overall?\n'

In [267]:
RANDOMIZE_WORD_PROMPT ="""
"Variables: Let V2 = health condition; X = maternal smoking status; V3 = infant's birth weight; Y = infant mortality.\nInfo: The overall probability of smoking mother is 87%. For infants with nonsmoking mothers, the probability of high infant mortality is 54%. For infants with smoking mothers, the probability of high infant mortality is 31%.\nQuestion: Is high infant mortality less likely than low infant mortality overall?\n"
 -> 
"Variables: Let V2 = health condition; X = maternal smoking status; V3 = infant's birth weight; Y = ice cream sales.\nInfo: The overall probability of smoking mother is 87%. For infants with nonsmoking mothers, the probability of high ice cream sales is 54%. For infants with smoking mothers, the probability of high cream sales is 31%.\nQuestion: Is high ice cream sales less likely than low ice cream sales overall?\n"

'Variables: Let V2 = residency status; X = gender; V3 = department competitiveness; Y = admission status.\nInfo: For individuals who are not male and applicants to a non-competitive department, the probability of admission acceptance is 64%. For individuals who are not male and applicants to a competitive department, the probability of admission acceptance is 39%. For individuals who are male and applicants to a non-competitive department, the probability of admission acceptance is 65%. For individuals who are male and applicants to a competitive department, the probability of admission acceptance is 39%. For individuals who are not male and out-of-state residents, the probability of competitive department is 51%. For individuals who are not male and in-state residents, the probability of competitive department is 21%. For individuals who are male and out-of-state residents, the probability of competitive department is 78%. For individuals who are male and in-state residents, the probability of competitive department is 47%. The overall probability of in-state residency is 93%.\nQuestion: Does gender positively affect admission status through department competitiveness?\n'
->
“Variables: Let V2 = residency status; X = gender; V3 = department competitiveness; Y = mortality rate.\nInfo: For individuals who are not male and applicants to a non-competitive department, the probability of mortality is 64%. For individuals who are not male and applicants to a competitive department, the probability of mortality is 39%. For individuals who are male and applicants to a non-competitive department, the probability of mortality is 65%. For individuals who are male and applicants to a competitive department, the probability of mortality is 39%. For individuals who are not male and out-of-state residents, the probability of competitive department is 51%. For individuals who are not male and in-state residents, the probability of competitive department is 21%. For individuals who are male and out-of-state residents, the probability of competitive department is 78%. For individuals who are male and in-state residents, the probability of competitive department is 47%. The overall probability of in-state residency is 93%.\nQuestion: Does gender positively affect mortality rate through department competitiveness?”.' 
"""

INSTRUCTION_PROMPT = """
-> (Change Y to a random name that doesn't make sense in Question and Info, and return the final paragraph similar to the previous examples) ->
"""

In [275]:
from openai import OpenAI
from constants import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY) 


def perturb(info, question, variables, temperature=1.0):
    
    QUESTION_PROMPT = 'Variables: ' + variables + '\n' + 'Info: ' + info + '\n' + 'Question: ' + question + '\n'
    
    prompt = RANDOMIZE_WORD_PROMPT + QUESTION_PROMPT + INSTRUCTION_PROMPT

    completion = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
    )
    
    model_answer = completion.choices[0].message.content.lower()
    return model_answer


answer = perturb(info, question, variables)
answer


'“variables: let x = eating citrus; v2 = vitamin c; y = balloon sales.\ninfo: we know that citrus intake causes sufficient vitamin c, and we know that sufficient vitamin c causes balloon sales.\nquestion: would the patient experience balloon sales if citrus intake instead of absence of citrus?”'

In [278]:
import re


def extract_variables_info_question(text):
    # Define regular expressions to match "variables," "info," and "question"
    variables_pattern = r'variables:\s*(.*?)\ninfo:'
    info_pattern = r'info:\s*(.*?)\nquestion:'
    question_pattern = r'question:\s*(.*?)$'
    
    # Extract variables, information, and question using regex
    variables_match = re.search(variables_pattern, text, re.DOTALL)
    info_match = re.search(info_pattern, text, re.DOTALL)
    question_match = re.search(question_pattern, text, re.DOTALL)
    
    # Get the matched groups or return None if not found
    variables = variables_match.group(1).strip() if variables_match else None
    info = info_match.group(1).strip() if info_match else None
    question = question_match.group(1).strip() if question_match else None
    
    return variables, info, question

variables, info, question = extract_variables_info_question(answer)

In [279]:
print(variables)
print(info)
print(question)

let x = eating citrus; v2 = vitamin c; y = balloon sales.
we know that citrus intake causes sufficient vitamin c, and we know that sufficient vitamin c causes balloon sales.
would the patient experience balloon sales if citrus intake instead of absence of citrus?”


In [287]:

# Apply perturbations
for i in range(0, 1000):

    variables = df_cladder.iloc[i]['reasoning']['step0']
    info = df_cladder.iloc[i]['info']
    question = df_cladder.iloc[i]['question']

    print(i, ': Info: ', info)
    print(i, ': Question: ', question)
    
    answer = perturb(info, question, variables)
    variables, info, question = extract_variables_info_question(answer)

    df_cladder.at[i, 'anticommonsensical_info'] = info
    df_cladder.at[i, 'anticommonsensical_question'] = question

    print(i, ': Anticommonsensical info: ', info)
    print(i, ': Anticommonsensical question: ', question)

    print('-------------------------------------------')

    print(i)

0 : Info:  The overall probability of manager signing the termination letter is 39%. For managers who don't sign termination letters, the probability of employee being fired is 22%. For managers who sign termination letters, the probability of employee being fired is 60%.
0 : Question:  Is employee being fired less likely than employee not being fired overall?
0 : Anticommonsensical info:  the overall probability of manager signing the termination letter is 39%. for managers who don't sign termination letters, the probability of tulip festival is 22%. for managers who sign termination letters, the probability of tulip festival is 60%.
0 : Anticommonsensical question:  is tulip festival less likely than tulip festival not happening overall?"
-------------------------------------------
0
1 : Info:  For unvaccinated individuals, the probability of smallpox survival is 35%. For vaccinated individuals, the probability of smallpox survival is 40%.
1 : Question:  Does vaccination status posit

In [289]:
df_cladder.to_pickle('../data/cladder/anticommonsensical-data.pkl')